In [1]:
# time.measure.start.py
from time import sleep, time

def f():
    sleep(.3)
    
def g():
    sleep(.5)
    
t = time()
f()
print('f took:', time() - t)

t = time()
g()
print('g took:', time() - t)

f took: 0.3046743869781494
g took: 0.5127854347229004


In [2]:
# improvising the above code. Reducing repeating logic to calculate time.
# time.measure.dry.py

from time import sleep, time

def f():
    sleep(.3)

def g():
    sleep(.5)
    
def measure(func):
    t = time()
    func()
    print(func.__name__, 'took:', time() - t)
    
measure(f)
measure(g)

f took: 0.30739665031433105
g took: 0.5062332153320312


In [8]:
# improvising the above again. Simplifying the function names and allow to input value to the function
# time.measure.arguments.py

from time import sleep, time

def f(sleep_time=0.1):
    sleep(sleep_time)
    
def measure(func, *args, **kwargs):  # *args to take value for function as input and **kwargs to acept key=value 
    t = time()
    func(*args, **kwargs)
    print(func.__name__, 'took:', time() - t)
    
measure(f, sleep_time=0.3)
measure(f, 0.4)
measure(f)

f took: 0.30181455612182617
f took: 0.4075634479522705
f took: 0.11276745796203613


In [11]:
# building in calculation function into f itself.
# time.measure.deco1.py

from time import sleep, time

def f(sleep_time=0.1):
    sleep(sleep_time)
    
def measure(func):
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper

f = measure(f)  # decoration point

f(0.2)
f(sleep_time=0.3)
print(f.__name__)

f took: 0.20592737197875977
f took: 0.30672788619995117
wrapper


### Some samples with decorator

In [18]:
# decorators/syntax.py  # example. Do not execute.
def func(arg1, arg2, ...):
pass
func = decorator(func)
# is equivalent to the following:
@decorator
def func(arg1, arg2, ...):
pass

SyntaxError: invalid syntax (Temp/ipykernel_33524/1320929622.py, line 2)

In [ ]:
# decorators/syntax.py
def func(arg1, arg2, ...):
pass
func = deco1(deco2(func))
# is equivalent to the following:
@deco1
@deco2
def func(arg1, arg2, ...):
pass

# order of applying decorators matters.

In [25]:
# time.measure.deco2.py

from time import sleep, time
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper

@measure
def f(sleep_time=0.1):
    """I'm a cat. I love to sleep!"""
    sleep(sleep_time)
    
f(sleep_time=0.3)
f(0.3)
print(f.__name__,':',f.__doc__)

f took: 0.3002157211303711
f took: 0.31505465507507324
f : I'm a cat. I love to sleep!


In [55]:
# two.decorators.py

from time import sleep, time
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        result = func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
        return result
    return wrapper
        
def max_result(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        if result > 100:
            print('Resuls is too big ({0}). Max allowed is 100.'.format(result))
        return result
    return wrapper

@measure
@max_result
def cube(n):
    return n ** 3


#print(cube(3))
print(cube(5))

Resuls is too big (125). Max allowed is 100.
cube took: 0.0
125


### Decorators tutorial with some youtube tutorial

In [37]:
# Division based decorator example
# a function div will take 2 inputs. numerator should be the big number all the time.

def div(a, b):
    return a / b

def smart_div(func):
    def wrapper(a, b):
        if a < b:
            a, b = b, a
            return func(a, b)
    return wrapper

div = smart_div(div)  
div(5, 10)

2.0

In [78]:
# write a decorator to avoid division by 0 error

from functools import wraps

def zero_avoider(func):
    @wraps(func)
    def wrapper(a, b):
        if a == 0:
            a = 1
        if b == 0:
            b = 1
        return func(a, b)
    return wrapper

def zero_finder(func):
    @wraps(func)
    def wrapper(a, b):
        if a == 0 or b == 0:
            print('Division by zero error observed')
        return func(a, b)
    return wrapper

#div = zero_avoider(div)
#div = zero_finder(div)

@zero_finder
@zero_avoider
def div(a, b):
    return a / b

div(12, 0.0)

Division by zero error observed


12.0

In [82]:
# decorator tutorial

def outer_function():
    message = 'Hi'
    
    def inner_function():
        print(message)
    return inner_function

my_func = outer_function()
my_func()

Hi


In [99]:
# calculate time taken for cubes to be calculated and return warning if it exceeds the specified
# limit

from time import sleep, time
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        sleep(.3)  # cheating just to see the value
        result = func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
        return result
    return wrapper

def max_result(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        if result > 10000:
            print('Result is too big ({0}). Max allowed is 1000.'.format(result))
        return result
    return wrapper

@max_result
@measure
def cube(n):
    return n ** 3

print(cube(10))

cube took: 0.30658936500549316
1000


In [107]:
# decorators.factory.py
# we should be able to define a threshold to check and return if breached

from functools import wraps

def max_result(threshold):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            if result > threshold:
                print('Result too big ({0}). Max allowed is {1}.'.format(result, threshold))
            return result
        return wrapper
    return decorator
                
@max_result(10000)
def cube(n):
    return n ** 3

@max_result(100)
def mult(a, b):
    return a * b


mult(100, 100)

Result too big (10000). Max allowed is 100.


1000